<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/report/football_live_binaryclass_1Xvs2_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Делает отчет по предиктам в мультиклассе и визуализрует результат<br>
Важные переменные

In [ ]:
remove_list = ['A1relativ', 'A2relativ', 'DA1relativ', 'DA2relativ'] #Ненужные фичи для предикта
selected_model_version = 1 #6 #2 Номер модели
folded = True #True #False # Модель на фолдах или нет

### Project config

In [ ]:
try:
    import neptune
except:
    !pip install neptune >> None
    import neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [ ]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [ ]:
!pip install catboost >> None

In [ ]:
# install orca
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -q -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4 >> None

### Downloads

In [ ]:
#!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/...здесь линк на холдаут
#!unrar x ./data.rar ./ >> None
#!mv ./FirstTime ./Result2

In [ ]:
data_230623 = False
if data_230623:
    !wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/6vXdKjlPrjHYfw
    !unrar x ./data.rar ./ >> None
    !mv ./StatsByStrOutput ./Result2

    !wget -q -O ./Prem.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/j0KaPC-p-aqvPA
    !unrar x ./Prem.rar ./ >> None

    !wget -q -O ./index.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/uyP8DHKyS7xwvQ
    !unrar x ./index.rar ./ >> None

In [ ]:
data_230624 = False
if data_230624:
    !wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/jREDtq9zENy_Kg
    !unrar x ./data.rar ./ >> None
    !mv ./RealStatsOutput ./Result2

    !wget -q -O ./Prem.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/j0KaPC-p-aqvPA
    !unrar x ./Prem.rar ./ >> None

    !wget -q -O ./index.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/uyP8DHKyS7xwvQ
    !unrar x ./index.rar ./ >> None

In [ ]:
data_230718 = True
if data_230718:
    !wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/p-WPqPGtt3XAAg
    !unrar x ./data.rar ./ >> None
    !mv ./RealStatsOutput ./Result2

    !wget -q -O ./Prem.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/r-0ho4pGcDHB1w
    !unrar x ./Prem.rar ./ >> None

    !wget -q -O ./index.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Gzj4j6FAk_2kLA
    !unrar x ./index.rar ./ >> None

### Imports

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

In [ ]:
np.iinfo(np.int8).max, np.iinfo(np.int16).max, np.iinfo(np.int32).max

In [ ]:
np.finfo(np.float16).precision, np.finfo(np.float32).precision, np.finfo(np.float64).precision

In [ ]:
import os, psutil, time
from glob import glob
from tqdm import tqdm
import functools

In [ ]:
from catboost import CatBoost

### Code

#####Functions

In [ ]:
def define_files_list(date_start:str, folder:str, date_end = '') -> list:
    '''
    data_start, data_end
    folder: ./folder/
    '''
    if date_end == '':
        return sorted(
            [file
            for file in glob(folder + '*')
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start)]
            )
    else:
        return sorted(
            [file
            for file in glob(folder + '*')
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start) & \
                int(file.split('/')[-1].split('.csv')[0]) <= int(date_end)]
            )

In [ ]:
def create_id_dict(date_start, folder, cols, date_end = '') -> dict:
    '''
    '''
    return pd.concat(map(
        functools.partial(
            pd.read_csv, sep=';', usecols =  cols
                            ),
                    define_files_list(date_start, folder, date_end = date_end)
                    ), ignore_index=True).drop_duplicates(subset = ['Id'], keep = 'last').set_index('Id').to_dict(orient = 'index')

In [ ]:
def transform_dataset(data_df:pd.DataFrame, remain = []):
    new_match_vector = data_df['Id'] != data_df['Id'].shift(1)
    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df[data_df['Score1'].isna() & new_match_vector] = 0
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
    data_df[data_df['Score2'].isna() & new_match_vector] = 0
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

    data_df['Score_diff'] = data_df['Score1'].astype(np.int16) - data_df['Score2'].astype(np.int16)
    data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
    data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 75
    data_df.loc[data_df['A1'] >= 60, ['A1_scaled']] = (60 + (data_df['A1'] - 60) / 4) / 75
    data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 75
    data_df.loc[data_df['A2'] >= 60, ['A2_scaled']] = (60 + (data_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    new_match_vector5 = data_df['Id'] != data_df['Id'].shift(5)
    data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A1relativ']] = np.float32(0.0)
    data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
    data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
    data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A2relativ']] =  np.float32(0.0)
    data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
    data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 50
    data_df.loc[data_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (data_df['DA1'] - 40) / 3) / 100
    data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 50
    data_df.loc[data_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (data_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту
    data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA1relativ']] = np.float32(0.0)
    data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
    data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
    data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA2relativ']] = np.float32(0.0)
    data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
    data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    if 'DA1' not in remain:
        data_df = data_df.drop(['DA1', 'DA2'], axis = 1)
    if 'Minute' not in remain:
        data_df = data_df.drop(['Minute'], axis = 1)
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df[data_df['Pos1'].isna() & new_match_vector] = 0
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    data_df[data_df['Pos2'].isna() & new_match_vector] = 0
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    if 'Pos1' not in remain:
        data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df[data_df['Off1'].isna() & new_match_vector] = 0
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    data_df[data_df['Off2'].isna() & new_match_vector] = 0
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    if 'Off1' not in remain:
        data_df = data_df.drop(['Off1', 'Off2'], axis = 1)
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df[data_df['On1'].isna() & new_match_vector] = 0
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    data_df[data_df['On2'].isna() & new_match_vector] = 0
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    if 'On1' not in remain:
        data_df = data_df.drop(['On1', 'On2'], axis = 1)
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df[data_df['YC1'].isna() & new_match_vector] = 0
    data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    data_df[data_df['YC2'].isna() & new_match_vector] = 0
    data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    if 'YC1' not in remain:
        data_df = data_df.drop(['YC1', 'YC2'], axis = 1)
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df[data_df['RC1'].isna() & new_match_vector] = 0
    data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    data_df[data_df['RC2'].isna() & new_match_vector] = 0
    data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    if 'RC1' not in remain:
        data_df = data_df.drop(['RC1', 'RC2'], axis = 1)
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df[data_df['Sub1'].isna() & new_match_vector] = 0
    data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    data_df[data_df['Sub2'].isna() & new_match_vector] = 0
    data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    if 'Sub1' not in remain:
        data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df[data_df['Cor1'].isna() & new_match_vector] = 0
    data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    data_df[data_df['Cor2'].isna() & new_match_vector] = 0
    data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    if 'Cor1' not in remain:
        data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
    data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2
    if 'P1' not in remain:
        data_df = data_df.drop(['P1', 'P2'], axis = 1)
    print('12. Линии посчитаны...')
    if 'Pen1' not in remain:
        data_df = data_df.drop(['Pen1', 'Pen2'], axis = 1)
    if 'Active' not in remain:
        data_df = data_df.drop(['Active'], axis = 1)
    return data_df

In [ ]:
def download_folded_model(model_number = selected_model_version, api_key = api_key):
    model_version_parameters = dict(
            project = 'scomesse/football',
            model = 'FOOT-LIVEBC2',
            api_token = api_key,
            with_id = 'FOOT-LIVEBC2-' + str(model_number)
                            )
    model_version = neptune.init_model_version(**model_version_parameters)
    if not os.path.exists('./models'):
        os.makedirs('./models')
    model_version_structure = model_version.get_structure()
    if 'description' in model_version_structure:
        folds_quantity = model_version['description/fold_quantity'].fetch()
        if 'list_of_runs' in model_version_structure['description']:
            print('SEE RUNS RESULTS: ', model_version['description/list_of_runs'].fetch())
    if 'models' in model_version_structure:
        for model_name in model_version_structure['models'].keys():
            model_version[f'models/{model_name}'].download(f"./models/{model_name}")
    else:
        print(f'no models in this version')
        return
    model_version.stop()
    return folds_quantity

In [ ]:
def restore_predict(X, folds_quantity, folded = folded, foldx = [2]):
    if folded:
        preds = sum(
            CatBoost().load_model(
                f'./models/booster_{kfold_num}.model'
                    ).predict(
                        X,
                        prediction_type="Probability"
                        )
            for kfold_num in range(folds_quantity)
                                ) / folds_quantity

    else:
        booster = CatBoost()
        booster.load_model('./booster.model')
        preds = booster.predict(X, prediction_type="Probability")

    return preds

#### Predefine data

In [ ]:
import datetime
data_types_dict = {
    'Id':np.int32,
    'StatTime':str,
    'Minute':np.int8,
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8,
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8,
    'Time':np.datetime64,
    'Minute.1':np.int8,
    'RawTime':np.datetime64,
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8,
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16,
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16
}

In [ ]:
k_cols = [
    'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2'
]

In [ ]:
drop_cols = [
    #'StatTime',
    'Comment', 'D', 'I', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'Period', 'Periods', 'Serve'
]

#### Load Parameters

In [ ]:
info_cols = ['Id', 'Result1', 'Result2']
prem_cols = ['Id', 'P1', 'PX', 'P2']
res_cols = [cols for cols in data_types_dict if cols not in drop_cols]

info_folder = './Index/'
prem_folder = './Prem/'
data_folder = './Result2/'

In [ ]:
### собрать датасет с даты:
year_start = '2023'
month_start = '01'
day_start = '01'
### до даты
year_end = ''
month_end = ''
day_end = ''
date_start = year_start + month_start + day_start
date_end = year_end + month_end + day_end

#### Create DataFrame

In [ ]:
info_dict = create_id_dict(date_start, info_folder, info_cols)
prem_dict = create_id_dict(date_start, prem_folder, prem_cols)
if len(prem_dict) != len(info_dict):
    print('Нет в info: ', set(prem_dict) - set(info_dict))
    print('Нет в prem: ', set(info_dict) - set(prem_dict))

In [ ]:
#files_list =['./Result2/' + str(id) + '.csv'
#    for id in set(info_dict)
#    if os.path.exists('./Result2/' + str(id) + '.csv')]

files_list = []
for folder in glob('./Result2/*'):
    files_list += glob(folder + '/*')

In [ ]:
time_point1 = time.time()
data_df = pd.concat(map(functools.partial(
    pd.read_csv,
    sep = ';',
    #parse_dates = ['StatTime'],
    usecols = res_cols
                                    ), files_list), ignore_index=True)
time_point2 = time.time()
print(f'create dataframe: {time_point2 - time_point1} sec.')
#---------------
process = psutil.Process(os.getpid())
print(round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

data_df[k_cols] = data_df[k_cols].fillna(0).values # Заполняем пропущенные лайв кэфы 1
new_match_vector = data_df['Id'] != data_df['Id'].shift(1) # Отмечаем переходы между матчами в колонках
data_cols = list(set(res_cols) - set(k_cols)) # Отбирвем колонки для заполнения пропусков метдом ffill
for col in data_cols:
    data_df.loc[data_df[col].isna() & new_match_vector, col] = 0 # Сами переходы с пропусками заполняем 0
data_df = data_df.fillna(method = 'ffill')
################ Записываем файл с временными метками и сбрасываем время матчей
#(data_df['Id'].astype(str) + ':' + data_df['StatTime'].astype(str).str.split(
#        ' ', expand=True)[0]
#        ).drop_duplicates().str.split(':', expand = True).rename(
#                                                columns = {0:'Id', 1:'StatTime'}
#                                                            ).to_csv('./time.csv', index = False)
#data_df = data_df.drop(['StatTime'], axis = 1)
############################################### Присоединяем результаты и кэфы
data_df[['Result1', 'Result2']] = [
    [info_dict[id]['Result1'], info_dict[id]['Result2']]
    if id in info_dict else [np.nan, np.nan]
    for id in tqdm(data_df['Id'].values, total = len(data_df))
    ]
data_df[['P1', 'PX', 'P2']] = [
    [prem_dict[id]['P1'], prem_dict[id]['PX'], prem_dict[id]['P2']]
    if id in prem_dict else [np.nan, np.nan, np.nan]
    for id in tqdm(data_df['Id'].values, total = len(data_df))
    ]
data_df = data_df[~(data_df["Result1"].isna() | data_df["Result1"].isna())]

print('\n')
#---------------
process = psutil.Process(os.getpid())
print(round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

In [ ]:
data_df.loc[new_match_vector & (data_df['P1'].isna()), ['P1']] = data_df['W1'][new_match_vector & (data_df['P1'].isna())]
data_df.loc[new_match_vector & (data_df['P2'].isna()), ['P2']] = data_df['W2'][new_match_vector & (data_df['P2'].isna())]
data_df.loc[new_match_vector & (data_df['PX'].isna()), ['PX']] = data_df['WX'][new_match_vector & (data_df['PX'].isna())]
data_df['P1'] = data_df['P1'].fillna(method = 'ffill').values
data_df['PX'] = data_df['PX'].fillna(method = 'ffill').values
data_df['P2'] = data_df['P2'].fillna(method = 'ffill').values

In [ ]:
data_df = transform_dataset(data_df, #)
    remain = ['StatTime', 'Minute', 'Score1', 'Score2', 'Result1', 'Result2', 'P1', 'PX', 'P2'])

In [ ]:
# Из-за того что некоторые кэфы были пропущены, и заменены на нули в препроцессинге на их месте появились inf, удаляем
print('P1 -inf & P2 -inf: ', (data_df['P1_transformed'] == -np.inf).sum(), (data_df['P2_transformed'] == -np.inf).sum())

#data_df = data_df[~(data_df['P1_transformed'] == -np.inf)]
#data_df = data_df[~(data_df['P2_transformed'] == -np.inf)]


In [ ]:
#Там где забито болше внутри матча чем в итоге, пока не удаляем
print('Score-Result error 1&2: ', ((~((data_df['Result1'] - data_df['Score1']) >= 0))).sum(), (~((data_df['Result1'] - data_df['Score1']) >= 0)).sum())
#data_df = data_df.loc[(data_df['Result1'] - data_df['Score1']) >= 0]
#data_df = data_df.loc[(data_df['Result2'] - data_df['Score2']) >= 0]


#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

In [ ]:
data_df.head(100)

In [ ]:
#data_df['NewTime'] = data_df['StatTime'].copy(deep = True)
#data_df['StatTime']  = pd.to_datetime([new_date.split(' ')[0] for new_date in tqdm(data_df['NewTime'].values, total = len(data_df))], dayfirst = True)
#data_df['StatTime'] = data_df['StatTime'].fillna(method = 'bfill')

In [ ]:
time_point1 = time.time()
data_df['StatTime'] = pd.to_datetime(data_df['StatTime'], dayfirst=True)
#data_df = data_df.sort_values(by = ['StatTime', 'Id', 'Minute'])
data_df = data_df.sort_values(by = ['StatTime'])
#data_df = data_df.drop('NewTime', axis = 1)
time_point2 = time.time()
print(f'sort by date: {time_point2 - time_point1} sec.')
#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

In [ ]:
data_df.info()

In [ ]:
data_df.describe()

In [ ]:
selected_cols = [item for item in data_df.columns[23:] if item not in remove_list]
#selected_cols = [item for item in data_df.columns[23:]]

In [ ]:
X = data_df[selected_cols].values
y = ((data_df['Result1'] - data_df['Result2'] >  0) * 1).values
line = data_df[['W2', 'X1']].values

#### Download models and restore predicts

In [ ]:
folds_quantity = download_folded_model()

In [ ]:
preds = restore_predict(X, folds_quantity)

#### Report

In [ ]:
data_df[['A_prob', 'H_prob']] = preds

In [ ]:
report_cols = ['Id', 'StatTime', 'Minute', 'Active.1', 'Score1', 'Score2', 'Result1', 'Result2', 'W1', 'X2', 'H_prob', 'A_prob']

In [ ]:
data_df[report_cols].head(100)

In [ ]:
#data_df[(data_df['StatTime'] > '2023-05-15 23:59:59') &\
#                     (data_df['StatTime'] < '2023-05-16 12:00:59')].to_csv('./result_report.csv', index = None)

In [ ]:
#data_df

#### Upload to Yandex

In [ ]:
data_df[report_cols].to_csv('./multiclass_probabilities.csv', index = False)

In [ ]:
#data_df.to_csv('./multiclass_dataframe.csv', index = False)

In [ ]:
try:
    import yadisk
except:
    !pip install yadisk >> None
    import yadisk
def get_credential2(frmwork = 'yandex token'):
    with open('cred.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw
_, api_key2 = get_credential2()
y = yadisk.YaDisk(token = api_key2)
print('Token exists: ', y.check_token())

In [ ]:
date_upload = 20230718

In [ ]:
print(y.mkdir(f'disk:/sport_prediction/calcio_2021/uploads/{date_upload}'))

In [ ]:
with open('/content/multiclass_probabilities.csv', 'rb') as f:
    y.upload(f, f'disk:/sport_prediction/calcio_2021/uploads/{date_upload}/multiclass_probabilities_folded.csv')

### visualize results

#### assign data

In [ ]:
y_dict = {'validation':((data_df['Result1'] - data_df['Result2'] >  0) * 1).values}
line_dict = {'validation':data_df[['X2', 'W1']].values}
active_rows_dict = {'validation':\
                    ((data_df['W1'] > 1) & \
                    (data_df['X2'] > 1) & \
                    (data_df['Active.1'] > 0)).values
                    }
preds_dict = {'validation':preds}

#### imports

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

#### functions

In [ ]:
def calculate_multiclass(probability_2x:np.array, line_2x:np.array):

    probline = probability_2x * line_2x
    best_odd_result = np.argmax(probline, axis = 1)
    best_odd_float = np.take_along_axis(probline, best_odd_result.reshape(-1, 1), axis = 1)
    return {
        'argmax':best_odd_result,
        'float':best_odd_float[:,0],
    }

In [ ]:
data_df[['A_prob', 'H_prob']].values[data_df['Active.1'].values]

In [ ]:
def plot_ga_equity_separate(
    sel_best_dict = {'home':1.21, 'away':1.21},
    y = ((data_df['Result1'] - data_df['Result2'] >=  0) * 1).values,
    line = data_df[['W2', 'X1']].values,
    predict = data_df[['A_prob', 'H_prob']].values,
    active_row = data_df['Active.1'].astype(bool).values,
    x_date = np.empty(()),
    neptune_model_version = f'FOOT-LIVEBC2-{selected_model_version}', data_split = 'holdout'):
    hda_dict = {'all':{'ltype':'solid', 'colors':'rgb(93, 164, 214)', 'line_width':3},
                'home':{'ltype':'dot', 'colors':'rgb(255, 144, 14)', 'line_width':1},
                'away':{'ltype':'dot', 'colors':'rgb(255, 65, 54)', 'line_width':1}}
    layout_dict = {}
    data_list = []
    preds_int, preds_float = calculate_multiclass(
        predict[active_row],
        line[active_row]).values() # size = dsar

    ################
    threshold_vector_dict = {
        'home':(sel_best_dict['home'] <= preds_float) & (preds_int == 1), #size = dsar
        'away':(sel_best_dict['away'] <= preds_float) & (preds_int == 0)  #size = dsar
    }
    threshold_vector_dict['all'] = threshold_vector_dict['home'] | threshold_vector_dict['away']
    ################
    y_true = (y[active_row] == preds_int)[threshold_vector_dict['all']] * 1 #y:thrdsar
    k_size = np.take_along_axis(
        line[active_row],
        preds_int.reshape(-1,1),
        axis = 1)[:, 0][threshold_vector_dict['all']] #k_size:thrdsar
    profit_size = (k_size - 1) / k_size #profit_size:thrdsar
    bet_qty = np.sum(threshold_vector_dict['all']) #bet_qty:dsar
    cumsum_dict = {}
    bet_size_dict = {}
    for hda, value_dict in hda_dict.items():
        if hda != 'all':
            bet_size_dict[hda] = k_size * threshold_vector_dict[hda][threshold_vector_dict['all']]
            cumsum_dict[hda] = (
                y_true * profit_size + (y_true - 1) / k_size
                                ) * threshold_vector_dict[hda][threshold_vector_dict['all']]
            data_list += [
                go.Scatter(
                    x = x_date[active_row][threshold_vector_dict['all']],
                    y = np.cumsum(cumsum_dict[hda]),
                    name = hda,
                    line = dict(color = value_dict['colors'], width = value_dict['line_width'], dash = value_dict['ltype'])
                            )]
    cumsum_dict['all'] = (y_true * profit_size + (y_true - 1) / k_size)
    data_list += [
        go.Scatter(
            x = x_date[active_row][threshold_vector_dict['all']],
            y = np.cumsum(cumsum_dict['all']),
            name = 'all',
            line = dict(color = hda_dict['all']['colors'], width = hda_dict['all']['line_width'], dash = hda_dict['all']['ltype'])
                    )]
    title_text = f'<b>Binary Classification</b> equity curve {data_split}<br>Model in neptune.ai: {neptune_model_version}<br>' + \
        f"thresholds: home={sel_best_dict['home']} | away={sel_best_dict['away']}<br>" + \
        f"ROI: {np.round(np.cumsum(cumsum_dict['all'])[-1] * 100 / np.sum(1/k_size), 4)}%" + \
        f' | Bet quantity: {bet_qty}<br>'
    dt_min = min(x_date[active_row])
    dt_max = max(x_date[active_row])
    layout_dict.update({
    'width':1200,
    'height':800,
    #'title_x':0.5,
    'title_text':title_text,
    'paper_bgcolor':'rgb(229, 237, 247)',
    'plot_bgcolor':'rgb(229, 237, 247)',
    'xaxis':{'range':[dt_min, dt_max]}
                    })
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout)

#### visualization

In [ ]:
sel_best_dict ={'home':1.345, 'away':1.345}
fig = plot_ga_equity_separate(
    x_date = data_df['StatTime'],
    sel_best_dict = sel_best_dict #{'home':1.1465, 'draw':1.1465, 'away':1.1465}
    )
image_name = './binary_all.jpeg'
#fig.write_image(image_name)
fig.show()

In [ ]:
#fig.show()

In [ ]:
fig.write_image(image_name)
#from IPython.display import Image, display
#display(Image(filename = image_name))

In [ ]:
with open(image_name, 'rb') as f:
    y.upload(f, f'disk:/sport_prediction/calcio_2021/uploads/{date_upload}/multiclass_probabilities_folded.jpeg')

In [ ]:
model_version = neptune.init_model_version(
    model = 'FOOT-LIVEBC2',
    project = 'scomesse/football',
    api_token = api_key, # your credentials
    with_id = f'FOOT-LIVEBC2-{selected_model_version}'
            )

model_version[f'equity_home={sel_best_dict["home"]}_away={sel_best_dict["away"]}_dd_{date_upload}'].upload(neptune.types.File.as_html(fig))
model_version.stop()

### Betting Strategies

##### Functions

In [ ]:
def bet_open(match_id_dict:dict, match_minute:int, strategy = '5minute', nbets = 3, last_type = None, current_prob = None, prob_thres = 1.05):
    '''
    '''
    if strategy == '5minute':
        if 'last_bet_minute' in match_id_dict:
            if (match_minute - match_id_dict['last_bet_minute']) > 4:
                return True
            else:
                return False
        else:
            return True
    elif strategy == 'nbets':
        if 'nbets' in match_id_dict:
            if match_id_dict['nbets'] >= nbets:
                return False
            else:
                return True
        else:
            return True
    elif strategy == 'changes':
        if 'last_type' in match_id_dict:
            if match_id_dict['last_type'] != last_type:
                return True
            elif (current_prob / match_id_dict['last_prob']) > prob_thres:
                return True
            else:
                return False
        else:
            return True

In [ ]:
def plot_strategy(
    data_df = data_df[['Id', 'StatTime', 'Minute','Result1', 'Result2', 'Active.1', 'W2', 'X1', 'A_prob', 'H_prob']],
    strategy_type = 'changes', #'nbets' #'5minute'
    neptune_model_version = f'FOOT-LIVEBC2-{selected_model_version}',
    data_split = 'validation',
    sel_best_dict = {'home':1.143, 'draw':1.143, 'away':1.143} #{'home':1.1465, 'draw':1.1465, 'away':1.1465}
):
    predict = data_df[['A_prob', 'H_prob']].values
    line = data_df[['W2', 'X1']].values
    y = ((data_df['Result1'] - data_df['Result2'] >=  0) * 1).values

    type2int = {1:'home' , 0:'away'}
    hda_dict = {'all':{'ltype':'solid', 'colors':'rgb(93, 164, 214)', 'line_width':3, 'order':2},
                'home':{'ltype':'dot', 'colors':'rgb(255, 144, 14)', 'line_width':1, 'order':0},
                'away':{'ltype':'dot', 'colors':'rgb(255, 65, 54)', 'line_width':1, 'order':1}}

    match_live_dict = {}
    output_list = []
    date_list = []
    bet_size_list = []
    preds_int, preds_float = calculate_multiclass(
                                        predict,
                                        line
                                    ).values()
    for pred_win, pred_prob, line, y_true, match_id, \
                    match_time, match_minute, active in tqdm(zip(
            preds_int,
            preds_float,
            line,
            y,
            data_df['Id'].values,
            data_df['StatTime'],
            data_df['Minute'].values,
            data_df['Active.1'].values
                                ), total = y.shape[0]):
        if active:
            if pred_prob > sel_best_dict[type2int[pred_win]]:
                if match_id not in match_live_dict:
                    match_live_dict[match_id] = {'nbets':0}

                if bet_open(match_live_dict[match_id], match_minute, strategy = strategy_type,
                            nbets = 3,
                            last_type = pred_win, current_prob = pred_prob, prob_thres = 1.01):
                    bet_size = 1 / line[pred_win]
                    if pred_win == y_true:

                        event_list = [0, 0, 0]
                        event_list[pred_win] =  bet_size * (line[pred_win] - 1)
                        output_list.append(event_list)
                        date_list.append(match_time.date())
                        bet_size_list.append(bet_size)
                        match_live_dict[match_id]['last_bet_minute'] = match_minute
                        match_live_dict[match_id]['nbets'] += 1
                        match_live_dict[match_id]['last_type'] = pred_win
                        match_live_dict[match_id]['last_prob'] = pred_prob
                    else:
                        event_list = [0, 0, 0]
                        event_list[pred_win] = -bet_size
                        output_list.append(event_list)
                        date_list.append(match_time.date())
                        bet_size_list.append(bet_size)
                        match_live_dict[match_id]['last_bet_minute'] = match_minute
                        match_live_dict[match_id]['nbets'] += 1
                        match_live_dict[match_id]['last_type'] = pred_win
                        match_live_dict[match_id]['last_prob'] = pred_prob
    layout_dict = {}
    data_list = []
    for hda, value_dict in hda_dict.items():
        if hda != 'all':
            data_list += [
                go.Scatter(
                    x = date_list,
                    y = np.cumsum(np.array(output_list)[:, value_dict['order']]),
                    name = hda,
                    line = dict(color = value_dict['colors'], width = value_dict['line_width'], dash = value_dict['ltype'])
                            )]

    cumsum_all = np.cumsum(np.sum(np.array(output_list), axis = 1))
    data_list += [
        go.Scatter(
            x = date_list,
            y = cumsum_all,
            name = 'all',
            line = dict(color = hda_dict['all']['colors'], width = hda_dict['all']['line_width'], dash = hda_dict['all']['ltype'])
                    )]
    title_text = f'<b>BinaryClass</b> equity curve for strategie <b><i>{strategy_type}</i></b>' + \
        f"<br>thresholds: home={sel_best_dict['home']} | away={sel_best_dict['away']}" + \
        f'<br>Model in neptune.ai: {neptune_model_version}<br>' + \
        f"ROI: {round(100 * cumsum_all[-1] / sum(bet_size_list), 2)}%" + \
        f' | Bet quantity: {len(output_list)}<br>'
    #dt_min = min(x_date[new_ard])
    #dt_max = max(x_date[new_ard])
    layout_dict.update({
    'width':1200,
    'height':800,
    #'title_x':0.5,
    'title_text':title_text,
    'paper_bgcolor':'rgb(229, 237, 247)',
    'plot_bgcolor':'rgb(229, 237, 247)',
    #'xaxis':{'range':[dt_min, dt_max]}
                    })
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout) #, match_live_dict

##### Plot

In [ ]:
#sel_best_dict = {'home':1.255, 'away':1.06}
for strategy_type in ['5minute', 'nbets', 'changes']:
    fig = plot_strategy(
        strategy_type = strategy_type, #'changes', #'nbets' #'5minute'
        sel_best_dict = sel_best_dict
        )
    fig.show()
    image_name = f'binary_folded_{strategy_type}.jpeg'
    fig.write_image('./' + image_name)

In [ ]:
image_name = 'binary_classification_folded_5minute.jpeg'
fig.write_image('./' + image_name)

In [ ]:
with open('./' + image_name, 'rb') as f:
    y.upload(f, f'disk:/sport_prediction/calcio_2021/uploads/{date_upload}/{image_name}')